<a href="https://colab.research.google.com/github/Ashliz1/NYC-Crime-Data---Group-10-/blob/main/term_project_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of New York Crime

### Author: (Group 10) Ashley Gomez, Sung Ik Park
### Date : December 8th, 2025

## Executive Summary

This project analyzes crime patterns in New York using the spotcrime dataset. The goal is to identify which areas appear relatively safer than others based on crime type, location, and time of day. Using Python (ADD MORE HERE!), we will review the data, summarize, and visualize to reveal trends/patterns across New York. The findings provide a basic overview of where crime clusters occur, offering insight for residents, policymakers, and anyone evaluating safety within New York.

## Table of Contents

1. Introduction
2. Problem Statement / Research Question
3. Data Description
4. Setup and Environment
5. Data Loading
6. Data Preparation
7. Model Planning
8. Model Building / Analysis
9. Discussion & Interpretation
10. Conclusion
11. References
12. Appendix

## Introduction

The project analyzes crime data in New York by identifying trends and patterns in where incidents occur most often, exploring how crime is spread across the city. The analysis focuses on three main aspects of data: location, crime type, and time of day. We use the SpotCrime dataset, which includes values such as location, timestamp, crime type, longitude, and latitude. Our first steps involve cleaning the data, creating summaries, and using Python to organize the data. Overall, the goal is to understand which areas are safer than others by providing an overview of crime patterns across New York.

## Problem Statement / Research Question

This project aims to determine which areas in New York appear safer or less safe based on recorded crime counts in the dataset. Understanding the overall distribution of crime can help people decide where to live and support decision-makers seeking a clearer view of safety conditions. The analysis also considers which crime types are most frequent and whether certain times of day show higher activity. It is expected that some areas will have noticeably higher incident counts than others and that specific crime types may appear more frequently depending on location and time. The approach uses simple descriptive methods, including grouping and counting incidents and creating visualizations, to highlight where crime is most concentrated across New York.

## Data Description

The data set contains individual crime incident reports from various locations across New York. Each row represents a single reported crime type, date, time, and geographic information such as county, city, ZIP code, address, and latitude/longitude. The dataset is provided in CSV format and contains a larger number of rows and columns. Data such as the timestamp column will be converted to a proper datetime format, and specific datasets may include missing values or duplicates. Although the data set includes several variables, the key focus will be on location, time, and crime type; the level of detail helps capture enough information to identify patterns/trends in crime activity across New York.

## Setup and Environment

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
import os

In [ ]:
data_url = "https://raw.githubusercontent.com/Ashliz1/NYC-Crime-Data---Group-10-/refs/heads/main/spotcrime.crime.ny.csv"
df = pd.read_csv(data_url)
df.head()

,_id,CrimeID,StateAbbreviated,State,CountyFips,CountyName,Latitude,Longitude,City,Zip,CBSACode,DMANameCBSA,CBSAStatus,CBSADivision,CBSADivisionTitle,TimeZone,CrimeType,CrimeAddress,CrimeTime
0,67deff0c92344d4401ae7df4,54aa76a5-415d-4c2c-a801-483fe9c4e267,NY,New York,36081,Queens,40.7468,-73.8605,Corona,11368,35620.0,"New York-Newark-Jersey City, NY-NJ-PA",Metropolitan Statistical Area,35614.0,"New York-Jersey City-White Plains, NY-NJ",EST,Theft,94-*** 64TH RD,2025-03-17T01:00:00.000Z
1,67deff0c92344d4401ae7df5,5a9770f1-ae80-400a-8051-6ff765f14f2a,NY,New York,36081,Queens,40.7468,-73.8605,Corona,11368,35620.0,"New York-Newark-Jersey City, NY-NJ-PA",Metropolitan Statistical Area,35614.0,"New York-Jersey City-White Plains, NY-NJ",EST,Theft,55-*** REAR 98TH PL,2025-03-16T23:00:00.000Z
2,67deff0c92344d4401ae7df6,2f104b66-df24-4cbb-b95f-b8ea82537bb5,NY,New York,36081,Queens,40.7468,-73.8605,Corona,11368,35620.0,"New York-Newark-Jersey City, NY-NJ-PA",Metropolitan Statistical Area,35614.0,"New York-Jersey City-White Plains, NY-NJ",EST,Assault,102-*** 42ND AVE,2025-03-16T20:00:00.000Z
3,67deff0d92344d4401ae7df7,cb4c504b-4998-48c3-9ca9-a158061d7375,NY,New York,36081,Queens,40.7468,-73.8605,Corona,11368,35620.0,"New York-Newark-Jersey City, NY-NJ-PA",Metropolitan Statistical Area,35614.0,"New York-Jersey City-White Plains, NY-NJ",EST,Theft,43-*** 108TH ST,2025-03-16T20:00:00.000Z
4,67deff0d92344d4401ae7df8,cd848914-6c49-45b0-aec4-2cb474b128ac,NY,New York,36081,Queens,40.7468,-73.8605,Corona,11368,35620.0,"New York-Newark-Jersey City, NY-NJ-PA",Metropolitan Statistical Area,35614.0,"New York-Jersey City-White Plains, NY-NJ",EST,Theft,94-*** 59TH AVE,2025-03-16T20:00:00.000Z


## Data Loading

In [ ]:
df.shape

(61824, 19)

### Functions

## Data Preparation

## Model Planning

## Model building / Analysis

## Discussion and Results

## Conclusion

## References

## Appendix